In [1]:
#Code Block 1
import nltk
import pandas as pd
import sys
import os
import io
import operator

from data_loader import DataLoader

from nltk.tokenize import word_tokenize ,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk import pos_tag

import math

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

import networkx as nx

from collections import defaultdict
import string

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import copy
import itertools

import pke

from  itertools import chain

import random
random.seed(0)

import time

In [2]:
#Code Block 2
loader = DataLoader(DataLoader.data_path1)
table = loader.load_table()
# table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:14: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


In [3]:
#Code Block 3
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

# print(merged_sw)

In [4]:
#Code Block 4
#Create the review text dictionary
reviewText = {}
for i in table.index:
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)

In [5]:
#Code Block 5
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [6]:
#Code Block 6
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
# print(processed_reviewText)

In [7]:
#Code Block 7
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [8]:
#Code Block 8
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
# print(frequency)

In [9]:
#Code Block 9
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
# print(sorted_processed_reviewText[:3])

In [10]:
#Code Block 10
print(len(processed_reviewText[sorted_processed_reviewText[1][0]]))

1511


In [11]:
#Code Block 11
text = """"""
for i in reviewText[sorted_processed_reviewText[1][0]]:
    text+=i
# print(text)

In [12]:
#Code Block 12
# Baseline Models Cite from https://github.com/boudinfl/pke

In [13]:
#Code Block 13
start = time.time()
#TextRank
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)
end = time.time()
print(end - start)

[('sweet new adult love story', 0.01873074306463817), ('own little way', 0.01787713990583066), ('good new adult read', 0.01720642591694452), ('little more emotional', 0.016661953890430747), ('new adult book', 0.016143372941646507), ('little bit', 0.015503498780450842), ('my first erin mccarthy book', 0.015193289441674926), ('own little world', 0.014782913413971397), ('emotional new adult romance', 0.014368288213148221), ('little more', 0.014353641931933553)]
2.2809557914733887


In [14]:
#Code Block 14
start = time.time()
# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None,
                        stoplist=merged_sw)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = False # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)

print(keyphrases)

end = time.time()
print(end - start)

[('robin', 0.00016692776126888526), ('phoenix', 0.0002285632064340584), ('true believers series', 0.00037332690716510616), ('erin mccarthy', 0.0007281100511081922), ('true believers', 0.000792142882191785), ('phoenix knows robin', 0.001223850272677977), ('new adult', 0.0012330087855895913), ('believe', 0.0016422877181582354), ('book', 0.0017387092834082662), ('true', 0.0018192728076754574)]
4.630007743835449


In [15]:
#Code Block 15
start = time.time()
# 1. create a TfIdf extractor.
extractor = pke.unsupervised.TfIdf()

# 2. load the content of the document.
stoplist = list(string.punctuation)
stoplist += pke.lang.stopwords.get('en')
extractor.load_document(input=text,
                        language='en',
                        stoplist=merged_sw,
                        normalization=None)

# 3. select {1-3}-grams not containing punctuation marks as candidates.
extractor.candidate_selection(n=3)

# 4. weight the candidates using a `tf` x `idf`
df = pke.load_document_frequency_file(input_file='df.tsv.gz')
extractor.candidate_weighting(df=df)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)

end = time.time()
print(end - start)

[('robin', 473.90378771562575), ('phoenix', 404.1654376838949), ('book', 282.1233251283658), ('series', 215.55490009807727), ('story', 199.70527509086568), ('love', 183.85565008365413), ('like', 164.83610007500025), ('characters', 163.25113757427908), ('loved', 136.30677506201945), ('believe', 134.7218125612983)]
3.496669292449951


In [16]:
#Code Block 16
start = time.time()
# TopicRank
# initialize keyphrase extraction model, here TopicRank
extractor = pke.unsupervised.TopicRank()

# load text
extractor.load_document(input=text, language='en')

# keyphrase candidate selection, in the case of TopicRank: sequences of nouns
# and adjectives (i.e. `(Noun|Adj)*`)
extractor.candidate_selection()

# candidate weighting, in the case of TopicRank: using a random walk algorithm
extractor.candidate_weighting()

# N-best selection, keyphrases contains the 10 highest scored candidates as
# (keyphrase, score) tuples
keyphrases = extractor.get_n_best(n=10)

print(keyphrases)

end = time.time()
print(end - start)

[('robin', 0.04257842931879103), ('phoenix', 0.03774593671528739), ('book', 0.02269715602136298), ('story', 0.01653469865746098), ('true believer', 0.0162741744072179), ('characters', 0.015724353540855395), ('entire series', 0.014671122994158304), ('friend', 0.010583860377464727), ('love', 0.009111823065316186), ('sweet', 0.008079805440531326)]
18.57978367805481


In [17]:
#Code Block 17
# RAKE refers to https://github.com/fabianvf/python-rake

In [18]:
#Code Block 18
# RAKE Algorithm
def is_num(current_word):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False
        
def split_words(text):
    splitter = re.compile(r'(?u)\W+')
    
    words = []
    for singleWord in splitter.split(text):
        currentWord = singleWord.strip().lower()
        
        if currentWord != '' and not is_num(currentWord):
            words.append(currentWord)
            
    return words


def build_regex(stopwords):
    sw_regex_list = []
    for word in stopwords:
        word_regex = r'\b' + word + r'(?![\w-])'
        sw_regex_list.append(word_regex)
        
    return re.compile('(?u)' + '|'.join(sw_regex_list), re.IGNORECASE)


def generate_keywords(sentences, stopWordPattern, minCharacters, maxWords):
    phrases = []
    for s in sentences:
        tmp = re.sub(stopWordPattern, '|', s.strip())
        ps = tmp.split("|")
        
        for phrase in ps:
            phrase = phrase.strip().lower()
            
            if phrase != '' and len(phrase) >= minCharacters and len(phrase.split()) <= maxWords:
                phrases.append(phrase)
                
    return phrases


def word_scores(phraseList):
    frequency = {}
    degree = {}
    for phrase in phraseList:
        wordList = split_words(phrase)
        wordListLen = len(wordList)
        wordListDegree = wordListLen - 1
        for word in wordList:
            frequency.setdefault(word, 0)
            frequency[word] += 1
            degree.setdefault(word, 0)
            degree[word] += wordListDegree
    for i in frequency:
        degree[i] = frequency[i] + degree[i]
    score = {}
    for i in frequency:
        score.setdefault(i, 0)
        score[i] = degree[i] / (frequency[i] * 1.0)
    return score


def generate_candidate_keyword_scores(phraseList, wordScore, minFrequency):
    ckScore = {}

    counts = defaultdict(int)
    for p in phraseList:
        counts[p] += 1

    for phrase in phraseList:
        if counts[phrase] >= minFrequency:
            ckScore.setdefault(phrase, 0)
            wordList = split_words(phrase)
            candidateScore = 0
            for word in wordList:
                candidateScore += wordScore[word]
            ckScore[phrase] = candidateScore
    return ckScore

def Rake(text, minCharNum, maxWordNum, minFrequency):
        sentences = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s').split(text)

        phrases = generate_keywords(sentences, build_regex(merged_sw), minCharNum, maxWordNum)

        wordScores = word_scores(phrases)

        ck = generate_candidate_keyword_scores(phrases, wordScores, minFrequency)

        sortedKeywords = sorted(ck.items(), key=operator.itemgetter(1), reverse=True)
        
        return sortedKeywords

In [19]:
#Code Block 19
start = time.time()

keywords = Rake(text, 2, 4, 2)
print(keywords[:10])

end = time.time()
print(end - start)

[('typical college party girl', 9.99688013136289), ('new adult genre', 8.202836879432624), ('january 21st 2014', 8.166666666666666), ('penguin group usa', 7.875), ('intermittent explosive disorder', 7.65), ('new adult series', 7.630764807360553), ('new adult books', 7.518928833455613), ('new adult book', 7.413829787234043), ('nal / signet romance', 7.328205128205129), ('erin mccarthy brings', 7.29201680672269)]
0.257007360458374
